In [13]:
import pandas as pd
import pymc as pm
import numpy as np
import arviz as az

# Load the dataset
df = pd.read_csv("Heart.csv")

# Selecting features and target variable
features = ["Age", "Gender", "BloodPressure", "Cholesterol", "HeartRate", "QuantumPatternFeature"]
target = "HeartDisease"

X = df[features]
y = df[target]

# Standardizing the features for better MCMC performance
X_standardized = (X - X.mean()) / X.std()

# Converting to NumPy arrays
X_data = X_standardized.values
y_data = y.values

# Number of predictors
p = X_data.shape[1]

# Bayesian Logistic Regression using PyMC
with pm.Model() as model:
    # Priors for coefficients
    betas = pm.Normal("betas", mu=0, sigma=1, shape=p)
    intercept = pm.Normal("intercept", mu=0, sigma=1)
    
    # Logit link function
    logits = intercept + pm.math.dot(X_data, betas)
    p = pm.math.sigmoid(logits)
    
    # Likelihood
    y_obs = pm.Bernoulli("y_obs", p=p, observed=y_data)
    
    # MCMC Sampling
    trace = pm.sample(2000, tune=1000, target_accept=0.9, return_inferencedata=True)

# Display summary statistics of the posterior
print(az.summary(trace))


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [betas, intercept]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 5 seconds.


            mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
betas[0]   0.064  0.193  -0.312    0.417      0.002    0.002   11615.0   
betas[1]  -0.043  0.172  -0.343    0.311      0.002    0.002   12181.0   
betas[2]  -0.198  0.165  -0.498    0.126      0.002    0.001   12112.0   
betas[3]  -0.097  0.188  -0.447    0.252      0.002    0.002   10806.0   
betas[4]  -0.135  0.170  -0.438    0.192      0.002    0.002   11632.0   
betas[5]  -5.163  0.473  -6.105   -4.320      0.005    0.004    8420.0   
intercept  1.452  0.209   1.062    1.843      0.002    0.002    8695.0   

           ess_tail  r_hat  
betas[0]     5748.0    1.0  
betas[1]     5667.0    1.0  
betas[2]     6474.0    1.0  
betas[3]     6570.0    1.0  
betas[4]     5557.0    1.0  
betas[5]     5715.0    1.0  
intercept    6517.0    1.0  
